### Ejercicio 3.4 optimizado

Primero se calcula el tiempo sin optimizar como se muestra en la siguiente celda

In [ ]:
import random
import sys

def calc_pi(N):
    M = 0
    for i in range(N):
    # Simulate impact coordinates
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
    # True if impact happens inside the circle
        if x**2 + y**2 < 1.0:
            M += 1
    return 4 * M / N


#num_trials = 10**7
num_trials = int(sys.argv[1])

pi = calc_pi(num_trials)

print("\n \t Computing pi in serial: \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi))
print("Tiempo sin optimizar con %timeit:")
%timeit -r3 calc_pi(num_trials)

Código mejorado utilizando los array numpy y numba

In [ ]:
import numpy as np
from numba import njit
import sys

@njit
def calc_pi_numba(N):
    np.random.seed(0)  
    # Aqui fijo la semilla para reproducibilidad
    # Generar puntos aleatorios: 
    x = np.random.uniform(-1, 1, N)
    y = np.random.uniform(-1, 1, N)
    # Cuento los puntos que caen dentro del circulo 
    M = np.sum(x**2 + y**2 < 1.0)
    return 4 * M / N

# num_trials=100
num_trials = int(sys.argv[1])

# Ahora calculamos el numero pi
pi = calc_pi_numba(num_trials)

print("\n \t Computing pi in serial: \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi))
print("Tiempo optimizado con %timeit:")
%timeit -r3 calc_pi_numba(num_trials)

En la siguiente celda se calcula la aceleración usando la función time.time() y viendo la diferencia entre ambos códigos

In [ ]:
import time
import numpy as np

# Versión inicial (sin optimizar) con time.time()
start_time_initial = time.time()
num_trials = int(sys.argv[1])
pi_initial = calc_pi(num_trials) 
end_time_initial = time.time()
time_initial = end_time_initial - start_time_initial
print(f"Tiempo de ejecución sin optimizar con time.time(): {time_initial:.4f} segundos")

# Versión optimizada
start_time_optimized = time.time()
pi_optimized = calc_pi_numba(num_trials) 
end_time_optimized = time.time()
time_optimized = end_time_optimized - start_time_optimized
print(f"Tiempo de ejecución optimizada con time.time(): {time_optimized:.4f} segundos")

# Calculo la mejora de los tiempos haciendo la division entre el tiempo inicial y el optimizado
acceleration_factor = time_initial / time_optimized
print(f"El factor de aceleración es: {acceleration_factor:.2f}X")

# Especificar la cola utilizada
# Se la voy a pasar por argumento
queue_used = sys.argv[2]  
print(f"Resultados ejecutados en la cola: {queue_used}")

El resultado obtenido tras ejecutar en la cola de nikola ha sido el que comento a continuación:


 	 Computing pi in serial: 

	 For 10000000 trials, pi = 3.140941

Tiempo sin optimizar con %timeit:
4.55 s ± 11.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

 	 Computing pi in serial: 

	 For 10000000 trials, pi = 3.141290

Tiempo optimizado con %timeit:
91.5 ms ± 92.9 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
Tiempo de ejecución sin optimizar con time.time(): 4.6865 segundos
Tiempo de ejecución optimizada con time.time(): 0.0919 segundos
El factor de aceleración es: 51.00X
Resultados ejecutados en la cola: hpc-bio-nikola-cpu


Como podemos observar se ha obtenido un factor de aceleración de 51, lo cual es bastante considerable, tras haber aplicado arrays de numpy y decorando la función con numba